In [3]:
import logging
import argparse 
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()
%config Completer.use_jedi = False # make autocompletion works in jupyter

args = argparse.Namespace()
args.data_folder = './data-ignored/imdb/'
args.val_fraction = 0.25
args.vocab_size = 2500

Path(args.data_folder).mkdir(parents=True, exist_ok=True)

ds, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True, data_dir=args.data_folder)
train_ds_len= tf.data.experimental.cardinality(ds['train']).numpy()
test_ds_len= tf.data.experimental.cardinality(ds['test']).numpy() 
print(train_ds_len)
for d in ds['train'].take(1):
    print(d)
    
train_dataset = ds['train']
val_dataset = ds['test'].take(int(args.val_fraction * (train_ds_len + test_ds_len)))
test_dataset = ds['test'].skip(int(args.val_fraction * (train_ds_len + test_ds_len)))

2021-11-10 16:22:29,506 : INFO : No config specified, defaulting to first: imdb_reviews/plain_text
2021-11-10 16:22:29,507 : INFO : Load dataset info from ./data-ignored/imdb/imdb_reviews/plain_text/1.0.0
2021-11-10 16:22:29,510 : INFO : Reusing dataset imdb_reviews (./data-ignored/imdb/imdb_reviews/plain_text/1.0.0)
2021-11-10 16:22:29,510 : INFO : Constructing tf.data.Dataset for split None, from ./data-ignored/imdb/imdb_reviews/plain_text/1.0.0


25000
(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


### Baseline

In [37]:
import functools

@functools.lru_cache()
def get_encoder():
    encoder = TextVectorization(max_tokens=args.vocab_size)
    encoder.adapt(train_dataset.map(lambda text, label: text))
    return encoder

In [39]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

def baseline():
    encoder = get_encoder()
    print(encoder([i[0] for i in train_dataset.take(1)][0]))

baseline()

tf.Tensor(
[  11   14   34  412  384   18   90   28    1    8   33 1322    1   42
  487    1  191   24   85  152   19   11  217  316   28   65  240  214
    8  489   54   65   85  112   96   22    1   11   93  642  743   11
   18    7   34  394    1  170 2464  408    2   88 1216  137   66  144
   51    2    1    1   66  245   65    1   16    1    1    1    1 1426
    1    3   40    1 1579   17    1   14  158   19    4 1216  891    1
    8    4   18   12   14    1    5   99  146 1241   10  237  704   12
   48   24   93   39   11    1  152   39 1322    1   50  398   10   96
 1155  851  141    9], shape=(116,), dtype=int64)
